In [2]:
import pinecone
import os


from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone




pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment="gcp-starter",
)

embeddings = OpenAIEmbeddings()

vector_store = Pinecone.from_documents(docs, embeddings, index_name="recipes")



ImportError: cannot import name 'Pinecone' from 'pinecone' (c:\Users\dioxin\.conda\envs\nomadlang\Lib\site-packages\pinecone\__init__.py)